In [1]:
print("Multimedia Security Experiment 12")

Multimedia Security Experiment 12


In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import myipf # 前面的实验编写的函数
from matplotlib.gridspec import GridSpec
#from mpl_toolkits.mplot3d.axes3d import Axes3D

%matplotlib inline
plt.rcParams["font.sans-serif"] = ["SimHei"] # 用来正常显示中文
plt.rcParams["axes.unicode_minus"] = False # 用来正常显示负号
%config InlineBackend.figure_format = "svg"